In [3]:
import pandas as pd
import pyodbc

In [17]:
conn = pyodbc.connect(
    "DRIVER={MySQL ODBC 8.0 Unicode Driver};"
    "SERVER=localhost;"
    "PORT=3306;"
    "DATABASE=sql_project_1;"
    "USER=root;"
    "PASSWORD=Mymummy1@;"
    "OPTION=3;"
)

In [18]:
query = "SELECT * FROM retail_sales"
basic = pd.read_sql(query, conn)

c:\Users\Admin\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [19]:
len(basic)

476